<a href="https://colab.research.google.com/github/hmck8625/first/blob/master/tensorflow_text_clf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

2.2.0


In [0]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [0]:
# 単語を整数にマッピングする辞書
word_index = imdb.get_word_index()

# インデックスの最初の方は予約済み
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [0]:
# 入力の形式は映画レビューで使われている語彙数（10,000語）
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [7]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/40
30/30 [==============================] - 1s 24ms/step - loss: 0.6925 - accuracy: 0.5155 - val_loss: 0.6912 - val_accuracy: 0.7366
Epoch 2/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6888 - accuracy: 0.7224 - val_loss: 0.6858 - val_accuracy: 0.7342
Epoch 3/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6794 - accuracy: 0.7481 - val_loss: 0.6730 - val_accuracy: 0.7528
Epoch 4/40
30/30 [==============================] - 1s 18ms/step - loss: 0.6607 - accuracy: 0.7672 - val_loss: 0.6504 - val_accuracy: 0.7639
Epoch 5/40
30/30 [==============================] - 1s 19ms/step - loss: 0.6306 - accuracy: 0.7896 - val_loss: 0.6177 - val_accuracy: 0.7706
Epoch 6/40
30/30 [==============================] - 1s 19ms/step - loss: 0.5909 - accuracy: 0.8012 - val_loss: 0.5784 - val_accuracy: 0.7882
Epoch 7/40
30/30 [==============================] - 1s 18ms/step - loss: 0.5453 - accuracy: 0.8222 - val_loss: 0.5358 - val_accuracy: 0.8104
Epoch 8/40
30

In [8]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)

782/782 - 1s - loss: 0.3262 - accuracy: 0.8735
[0.3262373208999634, 0.8734800219535828]
